In [1]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, r2_score

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Read the dataset
dataSet  = pd.read_csv("DataSet/app-store-apple-data-set-10k-apps/AppleStore.csv")

dataSet.head()

Unnamed: 0         id                                         track_name  \
0           1  281656475                                    PAC-MAN Premium   
1           2  281796108                          Evernote - stay organized   
2           3  281940292    WeatherBug - Local Weather, Radar, Maps, Alerts   
3           4  282614216  eBay: Best App to Buy, Sell, Save! Online Shop...   
4           5  282935706                                              Bible   

   size_bytes currency  price  rating_count_tot  rating_count_ver  \
0   100788224      USD   3.99             21292                26   
1   158578688      USD   0.00            161065                26   
2   100524032      USD   0.00            188583              2822   
3   128512000      USD   0.00            262241               649   
4    92774400      USD   0.00            985920              5320   

   user_rating  user_rating_ver     ver cont_rating   prime_genre  \
0          4.0              4.5   6.3.5          4+         Games   
1          4.0              3.5   8.2.2          4+  Productivity   
2          3.5              4.5   5.0.0          4+       Weather   
3          4.0              4.5  5.10.0         12+      Shopping   
4          4.5              5.0   7.5.1          4+     Reference   

   sup_devices.num  ipadSc_urls.num  lang.num  vpp_lic  
0               38                5        10        1  
1               37                5        23        1  
2               37                5         3        1  
3               37                5         9        1  
4               37                5        45        1

In [3]:
pd.set_option('float_format', '{:f}'.format)

In [4]:
dataSet['prime_genre'].unique()

array(['Games', 'Productivity', 'Weather', 'Shopping', 'Reference',
       'Finance', 'Music', 'Utilities', 'Travel', 'Social Networking',
       'Sports', 'Business', 'Health & Fitness', 'Entertainment',
       'Photo & Video', 'Navigation', 'Education', 'Lifestyle',
       'Food & Drink', 'News', 'Book', 'Medical', 'Catalogs'],
      dtype=object)

In [5]:
#  Drop the index and currency columns
dataSet.drop(labels=['Unnamed: 0', 'currency'], axis=1, inplace=True)

# Add the get_dummies columns to the original Dataframe
dataSet = pd.concat([dataSet, pd.get_dummies(dataSet['prime_genre'], prefix_sep="genre", dummy_na=True)], axis = 1)

In [6]:
set(dataSet['cont_rating'])

{'12+', '17+', '4+', '9+'}

In [7]:
#  Converting content rating column to categorical values

rating = {'4+':'G' ,'9+':'PG', '12+':'PG-13', '17+':'R'}

dataSet['rating'] = dataSet['cont_rating'].map(rating)

dataSet = pd.concat([dataSet, pd.get_dummies(dataSet['rating'], prefix_sep="rating", dummy_na=True)], axis = 1)

In [8]:
# Function to get features and predictor
def getFeatures(dataSet, genre, features):
    genreDataset = dataSet[dataSet[genre] == 1]
    X = genreDataset[features]
    y = genreDataset['user_rating']
    return X, y

In [9]:
# Function to make models for multiple genres 
def makeModel_getCoef(featuresDF, predictor, prefix=''):
    
    # create training and testing variables
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # make a Linear Regression model
    np.random.seed(42)
    lr = LinearRegression()
    model = lr.fit(X_train, y_train)
    
    # get coefficients    
    print("Mean Square error is :", np.sqrt(mean_squared_error(y_test, lr.predict(X_test))))
    print("R^2 score is :", r2_score(y_test, lr.predict(X_test)))
    return pd.DataFrame({'variable': featuresDF.columns, f'{prefix}_coefficient': model.coef_}).sort_values(by=f'{prefix}_coefficient')

In [10]:
# Building a Linear regression model for apps of Games genre
features = ['size_bytes','price','rating_count_tot','rating_count_ver','user_rating_ver','sup_devices.num',
            'ipadSc_urls.num','lang.num', 'G', 'PG', 'PG-13', 'R']
X, y = getFeatures(dataSet,'Games', features)
gamesCoef = makeModel_getCoef(X,y,'Games')
gamesCoef.head(10)

Mean Square error is : 0.8438706169660499
R^2 score is : 0.6967382717000803


variable  Games_coefficient
11                 R          -0.107997
5    sup_devices.num          -0.001909
9                 PG          -0.000390
0         size_bytes          -0.000000
2   rating_count_tot           0.000000
3   rating_count_ver           0.000004
7           lang.num           0.007877
1              price           0.015435
8                  G           0.028169
6    ipadSc_urls.num           0.043136

In [11]:
# Building a Linear regression model for apps of Education genre

X, y = getFeatures(dataSet,'Education', features)
eduCoef = makeModel_getCoef(X, y, 'Education')
eduCoef.head(10)

Mean Square error is : 1.2490046506772712
R^2 score is : 0.2953436707139696


variable  Education_coefficient
8                  G              -0.401428
10             PG-13              -0.027721
3   rating_count_ver              -0.000013
0         size_bytes              -0.000000
2   rating_count_tot               0.000007
7           lang.num               0.000030
1              price               0.006234
5    sup_devices.num               0.018676
6    ipadSc_urls.num               0.117442
9                 PG               0.166177

In [12]:
# Building a Linear regression model for apps of Productivity genre

X, y = getFeatures(dataSet,'Productivity', features)
ProductivityCoef = makeModel_getCoef(X,y, 'Productivity')
ProductivityCoef.head(10)

Mean Square error is : 0.7191254289376766
R^2 score is : 0.3810064818489426


variable  Productivity_coefficient
11                 R                 -0.189647
6    ipadSc_urls.num                 -0.011973
7           lang.num                 -0.003613
2   rating_count_tot                 -0.000001
0         size_bytes                  0.000000
3   rating_count_ver                  0.000345
5    sup_devices.num                  0.009422
9                 PG                  0.014064
1              price                  0.017707
10             PG-13                  0.066831

In [13]:
# Building a Linear regression model for apps of Weather genre

X, y = getFeatures(dataSet,'Weather', features)
weatherCoef = makeModel_getCoef(X,y, 'Weather')
weatherCoef.head(10)

Mean Square error is : 1.641995397252162
R^2 score is : -0.337670339656867


variable  Weather_coefficient
8                  G            -0.741676
10             PG-13            -0.557325
6    ipadSc_urls.num            -0.074728
5    sup_devices.num            -0.035855
2   rating_count_tot            -0.000000
9                 PG            -0.000000
0         size_bytes             0.000000
3   rating_count_ver             0.000154
7           lang.num             0.016271
1              price             0.030134

In [14]:
# Building a Linear regression model for apps of Shopping genre

X, y = getFeatures(dataSet,'Shopping', features)
shoppingCoef = makeModel_getCoef(X,y, 'Shopping')
shoppingCoef.head(10)

Mean Square error is : 1.6229726959163282
R^2 score is : -0.009365562419494822


variable  Shopping_coefficient
9                 PG             -0.452477
8                  G             -0.235707
7           lang.num             -0.024112
2   rating_count_tot             -0.000001
1              price             -0.000000
0         size_bytes              0.000000
3   rating_count_ver              0.000018
5    sup_devices.num              0.009998
6    ipadSc_urls.num              0.115060
10             PG-13              0.273557

In [15]:
# Building a Linear regression model for apps of Reference genre

X, y = getFeatures(dataSet,'Reference', features)
referenceCoef = makeModel_getCoef(X,y, 'Reference')
referenceCoef.head(10)

Mean Square error is : 1.350416875528755
R^2 score is : 0.24555018439786847


variable  Reference_coefficient
9                 PG              -1.093778
11                 R              -0.025800
7           lang.num              -0.015823
0         size_bytes              -0.000000
2   rating_count_tot               0.000002
3   rating_count_ver               0.000004
1              price               0.033837
5    sup_devices.num               0.042361
6    ipadSc_urls.num               0.097159
10             PG-13               0.344979

In [16]:
# Building a Linear regression model for apps of Finance genre

X, y = getFeatures(dataSet,'Finance', features)
financeCoef = makeModel_getCoef(X,y, 'Finance')
financeCoef.head(10)

Mean Square error is : 1.1044804439534717
R^2 score is : 0.6097455353468515


variable  Finance_coefficient
10             PG-13            -0.264602
2   rating_count_tot            -0.000004
9                 PG             0.000000
0         size_bytes             0.000000
3   rating_count_ver             0.000185
7           lang.num             0.007986
1              price             0.035933
11                 R             0.040974
5    sup_devices.num             0.055262
6    ipadSc_urls.num             0.191501

In [17]:
# Building a Linear regression model for apps of Music genre

X, y = getFeatures(dataSet,'Music', features)
musicCoef = makeModel_getCoef(X,y, 'Music')
musicCoef.head(10)

Mean Square error is : 0.4539734402976819
R^2 score is : 0.07140208365150047


variable  Music_coefficient
11                 R          -0.612439
5    sup_devices.num          -0.027088
2   rating_count_tot          -0.000001
0         size_bytes          -0.000000
3   rating_count_ver           0.000098
7           lang.num           0.000488
1              price           0.007214
8                  G           0.047153
6    ipadSc_urls.num           0.048745
10             PG-13           0.271764

In [18]:
# Building a Linear regression model for apps of Utilities genre

X, y = getFeatures(dataSet,'Utilities', features)
utilitiesCoef = makeModel_getCoef(X,y, 'Utilities')
utilitiesCoef.head(10)

Mean Square error is : 0.9839532744516726
R^2 score is : 0.5919052241172785


variable  Utilities_coefficient
10             PG-13              -0.448063
8                  G              -0.202607
6    ipadSc_urls.num              -0.047836
1              price              -0.022542
2   rating_count_tot              -0.000002
0         size_bytes               0.000000
3   rating_count_ver               0.000003
7           lang.num               0.005247
5    sup_devices.num               0.013934
11                 R               0.016703

In [19]:
# Building a Linear regression model for apps of Travel genre

X, y = getFeatures(dataSet,'Travel', features)
travelCoef = makeModel_getCoef(X,y, 'Travel')
travelCoef.head(10)

Mean Square error is : 1.3688692256429174
R^2 score is : 0.1056497860484934


variable  Travel_coefficient
11                 R           -0.345968
1              price           -0.119816
8                  G           -0.023726
3   rating_count_ver           -0.000137
2   rating_count_tot           -0.000003
9                 PG            0.000000
0         size_bytes            0.000000
7           lang.num            0.015425
5    sup_devices.num            0.119764
6    ipadSc_urls.num            0.163816

In [20]:
# Building a Linear regression model for apps of Social Networking genre

X, y = getFeatures(dataSet,'Social Networking', features)
netCoef = makeModel_getCoef(X,y, 'Social Networking')
netCoef.head(10)

Mean Square error is : 1.1919814661770531
R^2 score is : 0.42151353105211065


variable  Social Networking_coefficient
11                 R                      -0.117229
9                 PG                      -0.049611
5    sup_devices.num                      -0.036184
3   rating_count_ver                      -0.000011
2   rating_count_tot                      -0.000001
0         size_bytes                       0.000000
7           lang.num                       0.006860
10             PG-13                       0.030291
6    ipadSc_urls.num                       0.031489
1              price                       0.072527

In [21]:
# Building a Linear regression model for apps of Sports genre

X, y = getFeatures(dataSet,'Sports', features)
sportsCoef = makeModel_getCoef(X,y, 'Sports')
sportsCoef.head(10)

Mean Square error is : 1.163934194132792
R^2 score is : 0.5038705811175832


variable  Sports_coefficient
9                 PG           -0.487747
6    ipadSc_urls.num           -0.062344
11                 R           -0.003513
0         size_bytes            0.000000
2   rating_count_tot            0.000007
3   rating_count_ver            0.000047
7           lang.num            0.000148
5    sup_devices.num            0.053131
8                  G            0.070127
1              price            0.119805

In [22]:
# Building a Linear regression model for apps of Business genre

X, y = getFeatures(dataSet,'Business', features)
BusinessCoef = makeModel_getCoef(X,y, 'Business')
BusinessCoef.head(10)

Mean Square error is : 0.9959919196180727
R^2 score is : 0.38953852064954264


variable  Business_coefficient
10             PG-13             -2.010015
1              price             -0.002020
2   rating_count_tot             -0.000005
9                 PG              0.000000
0         size_bytes              0.000000
3   rating_count_ver              0.000110
5    sup_devices.num              0.005853
7           lang.num              0.009601
6    ipadSc_urls.num              0.057325
4    user_rating_ver              0.379234

In [23]:
# Building a Linear regression model for apps of Health & Fitness genre

X, y = getFeatures(dataSet,'Health & Fitness', features)
Health_FitnessCoef = makeModel_getCoef(X,y, 'Health & Fitness')
Health_FitnessCoef.head(10)

Mean Square error is : 1.3216086400180342
R^2 score is : 0.04355862723484427


variable  Health & Fitness_coefficient
9                 PG                     -0.651859
8                  G                     -0.100995
5    sup_devices.num                     -0.002680
2   rating_count_tot                     -0.000000
0         size_bytes                      0.000000
3   rating_count_ver                      0.000026
7           lang.num                      0.020198
6    ipadSc_urls.num                      0.039367
1              price                      0.099672
10             PG-13                      0.156220

In [24]:
# Building a Linear regression model for apps of Entertainment genre

X, y = getFeatures(dataSet,'Entertainment', features)
EntertainmentCoef = makeModel_getCoef(X,y, 'Entertainment')
EntertainmentCoef.head(10)

Mean Square error is : 1.2812365454911345
R^2 score is : 0.15601403049540663


variable  Entertainment_coefficient
11                 R                  -0.149654
10             PG-13                  -0.143840
1              price                  -0.032200
5    sup_devices.num                  -0.027115
2   rating_count_tot                  -0.000000
0         size_bytes                   0.000000
3   rating_count_ver                   0.000073
7           lang.num                   0.009007
8                  G                   0.026698
6    ipadSc_urls.num                   0.031104

In [25]:
# Building a Linear regression model for apps of Photo & Video genre

X, y = getFeatures(dataSet,'Photo & Video', features)
PhotoCoef = makeModel_getCoef(X,y, 'Photo & Video')
PhotoCoef.head(10)

Mean Square error is : 0.970651462996341
R^2 score is : 0.5149223896794777


variable  Photo & Video_coefficient
11                 R                  -0.353335
8                  G                  -0.091516
0         size_bytes                   0.000000
2   rating_count_tot                   0.000000
3   rating_count_ver                   0.000028
7           lang.num                   0.003592
5    sup_devices.num                   0.011220
1              price                   0.018597
9                 PG                   0.028136
6    ipadSc_urls.num                   0.050129

In [26]:
# Building a Linear regression model for apps of Navigation genre

X, y = getFeatures(dataSet,'Navigation', features)
NavigationCoef = makeModel_getCoef(X,y, 'Navigation')
NavigationCoef.head(10)

Mean Square error is : 5.548532004613631
R^2 score is : -8.021599239918464


variable  Navigation_coefficient
10             PG-13               -0.821319
6    ipadSc_urls.num               -0.160352
5    sup_devices.num               -0.014504
3   rating_count_ver               -0.000690
2   rating_count_tot               -0.000039
0         size_bytes               -0.000000
9                 PG                0.000000
7           lang.num                0.059419
1              price                0.064761
8                  G                0.298459

In [27]:
# Building a Linear regression model for apps of Lifestyle genre

X, y = getFeatures(dataSet,'Lifestyle', features)
LifestyleCoef = makeModel_getCoef(X,y, 'Lifestyle')
LifestyleCoef.head(10)

Mean Square error is : 1.3832420249058524
R^2 score is : 0.2980874599561155


variable  Lifestyle_coefficient
11                 R              -0.465917
5    sup_devices.num              -0.055140
9                 PG              -0.018618
3   rating_count_ver              -0.000005
2   rating_count_tot              -0.000000
0         size_bytes               0.000000
7           lang.num               0.004818
1              price               0.008810
6    ipadSc_urls.num               0.092348
8                  G               0.150023

In [28]:
# Building a Linear regression model for apps of Food & Drink genre

X, y = getFeatures(dataSet,'Food & Drink', features)
FoodDrinkCoef = makeModel_getCoef(X,y, 'FoodDrink')
FoodDrinkCoef.head(10)

Mean Square error is : 2.65893816821997
R^2 score is : -1.4970155043437159


variable  FoodDrink_coefficient
11                 R              -1.038368
5    sup_devices.num              -0.069714
8                  G              -0.053146
3   rating_count_ver              -0.000070
2   rating_count_tot              -0.000018
9                 PG               0.000000
0         size_bytes               0.000000
1              price               0.016348
7           lang.num               0.025805
6    ipadSc_urls.num               0.105280

In [29]:
# Building a Linear regression model for apps of News genre

X, y = getFeatures(dataSet,'News', features)
NewsCoef = makeModel_getCoef(X,y, 'News')
NewsCoef.head(10)

Mean Square error is : 1.1886554972176748
R^2 score is : 0.2717000561516507


variable  News_coefficient
9                 PG         -0.225289
11                 R         -0.200481
7           lang.num         -0.046472
8                  G         -0.032088
5    sup_devices.num         -0.012443
2   rating_count_tot         -0.000002
0         size_bytes          0.000000
3   rating_count_ver          0.001236
6    ipadSc_urls.num          0.158314
1              price          0.248529

In [30]:
# Building a Linear regression model for apps of Book genre

X, y = getFeatures(dataSet,'Book', features)
BookCoef = makeModel_getCoef(X,y, 'Book')
BookCoef.head(10)

Mean Square error is : 1.795519887928648
R^2 score is : 0.20026321578235673


variable  Book_coefficient
9                 PG         -0.376483
5    sup_devices.num         -0.117121
10             PG-13         -0.052489
8                  G         -0.014790
7           lang.num         -0.001913
3   rating_count_ver         -0.000724
0         size_bytes         -0.000000
2   rating_count_tot          0.000023
1              price          0.058626
6    ipadSc_urls.num          0.123778

In [31]:
# Building a Linear regression model for apps of Medical genre

X, y = getFeatures(dataSet,'Medical', features)
MedicalCoef = makeModel_getCoef(X,y, 'Medical')
MedicalCoef.head(10)

Mean Square error is : 1.1706218088678642
R^2 score is : 0.2936312271149123


variable  Medical_coefficient
11                 R            -0.402312
10             PG-13            -0.182105
1              price            -0.011094
3   rating_count_ver            -0.000697
9                 PG             0.000000
0         size_bytes             0.000000
2   rating_count_tot             0.000008
5    sup_devices.num             0.034414
6    ipadSc_urls.num             0.035219
7           lang.num             0.129860

In [32]:
# Building a Linear regression model for apps of Catalogs genre

X, y = getFeatures(dataSet,'Catalogs', features)
CatalogsCoef = makeModel_getCoef(X,y, 'Catalogs')
CatalogsCoef.head(10)

Mean Square error is : 0.5348099496127974
R^2 score is : 0.0


variable  Catalogs_coefficient
6    ipadSc_urls.num             -0.102008
1              price             -0.098482
11                 R             -0.024528
10             PG-13             -0.012326
3   rating_count_ver             -0.003798
9                 PG              0.000000
0         size_bytes              0.000000
7           lang.num              0.000000
2   rating_count_tot              0.000817
8                  G              0.036854

In [33]:
from functools import reduce

In [34]:
coefDF = reduce(lambda left,right: pd.merge(left,right,on='variable'), [gamesCoef,eduCoef,ProductivityCoef,weatherCoef,shoppingCoef,referenceCoef,financeCoef,
                   musicCoef,utilitiesCoef,travelCoef,netCoef,sportsCoef,BusinessCoef,Health_FitnessCoef,
                   EntertainmentCoef,PhotoCoef,NavigationCoef,LifestyleCoef,FoodDrinkCoef,
                   NewsCoef,BookCoef,MedicalCoef,CatalogsCoef])

In [35]:
coefDF.set_index('variable', inplace=True)

In [36]:
coefDF

Games_coefficient  Education_coefficient  \
variable                                                     
R                         -0.107997               0.262972   
sup_devices.num           -0.001909               0.018676   
PG                        -0.000390               0.166177   
size_bytes                -0.000000              -0.000000   
rating_count_tot           0.000000               0.000007   
rating_count_ver           0.000004              -0.000013   
lang.num                   0.007877               0.000030   
price                      0.015435               0.006234   
G                          0.028169              -0.401428   
ipadSc_urls.num            0.043136               0.117442   
PG-13                      0.080219              -0.027721   
user_rating_ver            0.743916               0.593644   

                  Productivity_coefficient  Weather_coefficient  \
variable                                                          
R                                -0.189647             1.299001   
sup_devices.num                   0.009422            -0.035855   
PG                                0.014064            -0.000000   
size_bytes                        0.000000             0.000000   
rating_count_tot                 -0.000001            -0.000000   
rating_count_ver                  0.000345             0.000154   
lang.num                         -0.003613             0.016271   
price                             0.017707             0.030134   
G                                 0.108752            -0.741676   
ipadSc_urls.num                  -0.011973            -0.074728   
PG-13                             0.066831            -0.557325   
user_rating_ver                   0.366292             0.378137   

                  Shopping_coefficient  Reference_coefficient  \
variable                                                        
R                             0.414627              -0.025800   
sup_devices.num               0.009998               0.042361   
PG                           -0.452477              -1.093778   
size_bytes                    0.000000              -0.000000   
rating_count_tot             -0.000001               0.000002   
rating_count_ver              0.000018               0.000004   
lang.num                     -0.024112              -0.015823   
price                        -0.000000               0.033837   
G                            -0.235707               0.774600   
ipadSc_urls.num               0.115060               0.097159   
PG-13                         0.273557               0.344979   
user_rating_ver               0.339164               0.741663   

                  Finance_coefficient  Music_coefficient  \
variable                                                   
R                            0.040974          -0.612439   
sup_devices.num              0.055262          -0.027088   
PG                           0.000000           0.293522   
size_bytes                   0.000000          -0.000000   
rating_count_tot            -0.000004          -0.000001   
rating_count_ver             0.000185           0.000098   
lang.num                     0.007986           0.000488   
price                        0.035933           0.007214   
G                            0.223628           0.047153   
ipadSc_urls.num              0.191501           0.048745   
PG-13                       -0.264602           0.271764   
user_rating_ver              0.437326           0.341034   

                  Utilities_coefficient  Travel_coefficient  \
variable                                                      
R                              0.016703           -0.345968   
sup_devices.num                0.013934            0.119764   
PG                             0.633967            0.000000   
size_bytes                     0.000000            0.000000   
rating_count_tot              -0.000002           -0.000003   
rating_count_ver 

In [37]:
coefDF.T.sort_values(by='sup_devices.num')

variable                              R  sup_devices.num        PG  \
Book_coefficient               0.443762        -0.117121 -0.376483   
FoodDrink_coefficient         -1.038368        -0.069714  0.000000   
Lifestyle_coefficient         -0.465917        -0.055140 -0.018618   
Social Networking_coefficient -0.117229        -0.036184 -0.049611   
Weather_coefficient            1.299001        -0.035855 -0.000000   
Entertainment_coefficient     -0.149654        -0.027115  0.266797   
Music_coefficient             -0.612439        -0.027088  0.293522   
Navigation_coefficient         0.522859        -0.014504  0.000000   
News_coefficient              -0.200481        -0.012443 -0.225289   
Health & Fitness_coefficient   0.596634        -0.002680 -0.651859   
Games_coefficient             -0.107997        -0.001909 -0.000390   
Business_coefficient           1.529402         0.005853  0.000000   
Productivity_coefficient      -0.189647         0.009422  0.014064   
Shopping_coefficient           0.414627         0.009998 -0.452477   
Photo & Video_coefficient     -0.353335         0.011220  0.028136   
Utilities_coefficient          0.016703         0.013934  0.633967   
Education_coefficient          0.262972         0.018676  0.166177   
Medical_coefficient           -0.402312         0.034414  0.000000   
Reference_coefficient         -0.025800         0.042361 -1.093778   
Sports_coefficient            -0.003513         0.053131 -0.487747   
Finance_coefficient            0.040974         0.055262  0.000000   
Travel_coefficient            -0.345968         0.119764  0.000000   
Catalogs_coefficient          -0.024528         0.135704  0.000000   

variable                       size_bytes  rating_count_tot  rating_count_ver  \
Book_coefficient                -0.000000          0.000023         -0.000724   
FoodDrink_coefficient            0.000000         -0.000018         -0.000070   
Lifestyle_coefficient            0.000000         -0.000000         -0.000005   
Social Networking_coefficient    0.000000         -0.000001         -0.000011   
Weather_coefficient              0.000000         -0.000000          0.000154   
Entertainment_coefficient        0.000000         -0.000000          0.000073   
Music_coefficient               -0.000000         -0.000001          0.000098   
Navigation_coefficient          -0.000000         -0.000039         -0.000690   
News_coefficient                 0.000000         -0.000002          0.001236   
Health & Fitness_coefficient     0.000000         -0.000000          0.000026   
Games_coefficient               -0.000000          0.000000          0.000004   
Business_coefficient             0.000000         -0.000005          0.000110   
Productivity_coefficient         0.000000         -0.000001          0.000345   
Shopping_coefficient             0.000000         -0.000001          0.000018   
Photo & Video_coefficient        0.000000          0.000000          0.000028   
Utilities_coefficient            0.000000         -0.000002          0.000003   
Education_coefficient           -0.000000          0.000007         -0.000013   
Medical_coefficient              0.000000          0.000008         -0.000697   
Reference_coefficient           -0.000000          0.000002          0.000004   
Sports_coefficient               0.000000          0.000007          0.000047   
Finance_coefficient              0.000000         -0.000004          0.000185   
Travel_coefficient               0.000000         -0.000003         -0.000137   
Catalogs_coefficient             0.000000          0.000817         -0.003798   

variable                       lang.num     price         G  ipadSc_urls.num  \
Book_coefficient              -0.001913  0.058626 -0.014790         0.123778   
FoodDrink_coefficient          0.025805  0.016348 -0.053146         0.105280   
Lifestyle_coefficient          0.004818  0.008810  0.150023         0.092348   
Social Networking_coefficient  0.006860  0.072527  0.1